In [ ]:
%run oeai_py

In [ ]:
# Create an instance of OEAI class and set the platform ("Synapse" or "Fabric")
oeai = OEAI("Fabric")

In [ ]:
pip install paramiko

In [ ]:
pip install pysftp

In [ ]:
import pysftp
import paramiko
import os

In [ ]:
# CHANGE VALUES FOR YOUR KEY VAULT
keyvault = "INSERT_YOUR_KEYVAULT_NAME" # fully qualified for Fabric 
keyvault_linked_service = "INSERT_YOUR_KEYVAULT_LINKED_SERVICE_NAME" # linked service name for Synapse

# Synapse OEA environment path & secrets
bronze_path = oeai.get_secret(spark, "renaissance-bronze", keyvault_linked_service, keyvault)

In [ ]:
# SFTP details

#https://transfer.renaissance.com/ThinClient/WTM/public/index.html#/login


host = "sftp.renaissance.com"
port = 22  # SFTP usually runs on port 22
username = "YOUR_USERNAME"
password = "YOUR_PASSWORD"
remote_path = ""
local_path = "YOUR_LOCAL_PATH"

# Ensure the local directory exists
os.makedirs(local_path, exist_ok=True)

# Print the local directory path for debugging
print(f"Local directory path: {local_path}")

# Function to list and download files using paramiko
def download_files():
    transport = paramiko.Transport((host, port))
    transport.connect(username=username, password=password)
    sftp = paramiko.SFTPClient.from_transport(transport)
    
    try:
        sftp.chdir(remote_path)
        print("Files in the remote directory:")
        for filename in sftp.listdir():
            print(filename)  # Print the list of files in the remote directory
            if filename.endswith(".zip"):
                local_file_path = os.path.join(local_path, filename)
                print(f"Downloading {filename} to {local_file_path}")
                sftp.get(filename, local_file_path)
    finally:
        sftp.close()
        transport.close()

# Call the download function
download_files()

# List files in the local directory to verify download
print("Files in the local directory:")
local_files = [os.path.join(local_path, file) for file in os.listdir(local_path)]
for file in local_files:
    print(file)

# Check if the paths actually exist and construct the list of zip files
zip_files = []
for file in local_files:
    if file.endswith('.zip'):
        if os.path.exists(file):
            print(f"File exists: {file}")
            zip_files.append(file)
        else:
            print(f"File does not exist: {file}")

# Debug: print zip files to be loaded
print(f"ZIP files to be loaded: {zip_files}")

# Load the downloaded files into a DataFrame
if zip_files:
    df = spark.read.format("binaryFile").load(zip_files)
    # Show the DataFrame
    df.show()
else:
    print("No valid ZIP files found to load.")

In [ ]:
import os

local_path = bronze_path  # Ensure this is defined with the correct path

# Ensure the local directory exists
if not os.path.exists(local_path):
    print(f"Directory {local_path} does not exist.")
else:
    print(f"Files in the local directory {local_path}:")
    local_files = os.listdir(local_path)
    for file in local_files:
        print(file)


In [ ]:
import os
import zipfile
import shutil

# Define the local path and the specific file to load
local_path = bronze_path  # Ensure this is defined with the correct path
zip_file = os.path.join(local_path, 'YOUR_SCHOOL.zip')

# Ensure the local directory exists
if not os.path.exists(local_path):
    print(f"Directory {local_path} does not exist.")
else:
    print(f"Local directory path: {local_path}")

# Check if the ZIP file exists
if os.path.exists(zip_file):
    print(f"ZIP file exists: {zip_file}")
    
    # Create a directory for the extracted contents
    extracted_path = os.path.join(local_path, 'extracted')
    if os.path.exists(extracted_path):
        shutil.rmtree(extracted_path)
    os.makedirs(extracted_path)
    
    # Extract the ZIP file
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extracted_path)
    
    print(f"Extracted files to: {extracted_path}")
    
    # List the extracted files
    extracted_files = [os.path.join(extracted_path, file) for file in os.listdir(extracted_path)]
    for file in extracted_files:
        print(f"Extracted file: {file}")
    
    # Load the extracted CSV files into a Spark DataFrame
    for csv_file in extracted_files:
        if csv_file.endswith('.csv'):
            absolute_path = os.path.abspath(csv_file)
            file_url = f"file://{absolute_path}"
            print(f"Loading CSV file: {file_url}")
            
            # Confirm the file exists
            if os.path.exists(absolute_path):
                print(f"File exists: {file_url}")
                
                # Load the file using Spark
                df = spark.read.format("csv").option("header", "true").load(file_url)
                
                # Show the DataFrame
                df.show()
            else:
                print(f"File does not exist: {file_url}")
else:
    print(f"ZIP file does not exist: {zip_file}")


In [ ]:
import os
import zipfile
import shutil

# Define the local path and the specific file to load
local_path = bronze_path  # Ensure this is defined with the correct path
zip_file = os.path.join(local_path, 'YOUR_SCHOOL.zip')
output_path = os.path.join(local_path, 'output')  # Define the output path for saving the DataFrame

# Ensure the local directory exists
if not os.path.exists(local_path):
    print(f"Directory {local_path} does not exist.")
else:
    print(f"Local directory path: {local_path}")

# Check if the ZIP file exists
if os.path.exists(zip_file):
    print(f"ZIP file exists: {zip_file}")
    
    # Create a directory for the extracted contents
    extracted_path = os.path.join(local_path, 'extracted')
    if os.path.exists(extracted_path):
        shutil.rmtree(extracted_path)
    os.makedirs(extracted_path)
    
    # Extract the ZIP file
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extracted_path)
    
    print(f"Extracted files to: {extracted_path}")
    
    # List the extracted files
    extracted_files = [os.path.join(extracted_path, file) for file in os.listdir(extracted_path)]
    for file in extracted_files:
        print(f"Extracted file: {file}")
    
    # Load the extracted CSV files into a Spark DataFrame and write to output path
    for csv_file in extracted_files:
        if csv_file.endswith('.csv'):
            absolute_path = os.path.abspath(csv_file)
            file_url = f"file://{absolute_path}"
            print(f"Loading CSV file: {file_url}")
            
            # Confirm the file exists
            if os.path.exists(absolute_path):
                print(f"File exists: {file_url}")
                
                # Load the file using Spark
                df = spark.read.format("csv").option("header", "true").load(file_url)
                
                # Show the DataFrame
                df.show()
                
                # Write the DataFrame to the output directory
                output_csv_path = os.path.join(output_path, os.path.basename(csv_file))
                df.write.format("csv").option("header", "true").save(output_csv_path)
                print(f"DataFrame written to: {output_csv_path}")
            else:
                print(f"File does not exist: {file_url}")
else:
    print(f"ZIP file does not exist: {zip_file}")


In [ ]:
import os
import zipfile
import shutil

# Define the local path where ZIP files are located
local_path = bronze_path  # Ensure this is defined with the correct path

# Ensure the local directory exists
if not os.path.exists(local_path):
    print(f"Directory {local_path} does not exist.")
else:
    print(f"Local directory path: {local_path}")

# Function to extract ZIP files into folders named after the ZIP file
def extract_zip_files():
    # List all ZIP files in the local directory
    zip_files = [f for f in os.listdir(local_path) if f.endswith('.zip')]
    
    for zip_file in zip_files:
        zip_file_path = os.path.join(local_path, zip_file)
        
        # Create a directory named after the ZIP file (excluding the .zip extension)
        folder_name = os.path.splitext(zip_file)[0]
        extracted_path = os.path.join(local_path, folder_name)
        
        # Remove existing directory if it exists
        if os.path.exists(extracted_path):
            shutil.rmtree(extracted_path)
        
        # Create the new directory
        os.makedirs(extracted_path)
        
        # Extract the ZIP file into the created directory
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_path)
        
        print(f"Extracted {zip_file} to {extracted_path}")
        
        # List the extracted files
        extracted_files = [os.path.join(extracted_path, file) for file in os.listdir(extracted_path)]
        for file in extracted_files:
            print(f"Extracted file: {file}")

# Call the function to extract ZIP files
extract_zip_files()

# Optional: Load the extracted CSV files into a Spark DataFrame (example for one CSV file)
folder_name = 'YOUR_SCHOOL'  # Example folder name
csv_file = 'NAME.csv'  # Example CSV file name
csv_file_path = os.path.join(local_path, folder_name, csv_file)

# Check if the CSV file exists and load it into Spark DataFrame
if os.path.exists(csv_file_path):
    file_url = f"file://{os.path.abspath(csv_file_path)}"
    print(f"Loading CSV file: {file_url}")
    
    df = spark.read.format("csv").option("header", "true").load(file_url)
    
    # Show the DataFrame
    df.show()
else:
    print(f"CSV file does not exist: {csv_file_path}")


In [ ]:
import os
import zipfile
import shutil

# Define the local path where ZIP files are located
local_path = bronze_path  # Ensure this is defined with the correct path

# Ensure the local directory exists
if not os.path.exists(local_path):
    print(f"Directory {local_path} does not exist.")
else:
    print(f"Local directory path: {local_path}")

# Function to extract ZIP files into folders named after the ZIP file
def extract_zip_files():
    # List all ZIP files in the local directory
    zip_files = [f for f in os.listdir(local_path) if f.endswith('.zip')]
    
    for zip_file in zip_files:
        zip_file_path = os.path.join(local_path, zip_file)
        
        # Create a directory named after the ZIP file (excluding the .zip extension)
        folder_name = os.path.splitext(zip_file)[0]
        extracted_path = os.path.join(local_path, folder_name)
        
        # Remove existing directory if it exists
        if os.path.exists(extracted_path):
            shutil.rmtree(extracted_path)
        
        # Create the new directory
        os.makedirs(extracted_path)
        
        # Extract the ZIP file into the created directory
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_path)
        
        print(f"Extracted {zip_file} to {extracted_path}")
        
        # List the extracted files
        extracted_files = [os.path.join(extracted_path, file) for file in os.listdir(extracted_path)]
        for file in extracted_files:
            print(f"Extracted file: {file}")

# Call the function to extract ZIP files
extract_zip_files()

# Load the extracted CSV files into a Spark DataFrame (example for one CSV file)
folder_name = 'YOUR_SCHOOL'  # Example folder name
csv_file = 'AR_UK.csv'  # Example CSV file name
csv_file_path = os.path.join(local_path, folder_name, csv_file)

# Ensure the path is correctly formatted and check existence
absolute_csv_path = os.path.abspath(csv_file_path)

print(f"Loading CSV file: {absolute_csv_path}")

# Check if the CSV file exists and load it into Spark DataFrame
if os.path.exists(absolute_csv_path):
    file_url = f"file://{absolute_csv_path}"
    print(f"File exists: {file_url}")
    
    df = spark.read.format("csv").option("header", "true").load(file_url)
    
    # Show the DataFrame
    df.show()
else:
    print(f"CSV file does not exist: {absolute_csv_path}")

# Save the combined DataFrame as a Delta table
csv_path = bronze_path + "AR_UK.csv"
df.write.format("CSV").mode("overwrite").save(csv_path)

